In [1]:
import tensorflow as tf
import _pickle as cPickle # python3中cPickle包已被替代
import os
import numpy

CIFAR_DIR = './cifar-10-python/cifar-10-batches-py'
print(os.listdir(CIFAR_DIR))

['batches.meta', 'data_batch_1', 'data_batch_2', 'data_batch_3', 'data_batch_4', 'data_batch_5', 'readme.html', 'test_batch']


In [2]:
def load_data(filename):
    """read data from load file"""
    with open(filename, 'rb') as f:
        data = cPickle.load(f, encoding='iso-8859-1') # python3打开pickle数据需要指定编码
        return data['data'], data['labels']

In [ ]:
"""tensorflow calculate graph"""
# 为data(0-1)提供占位符 数据类型为float32 大小为[None, 3072] 行数暂定(计算梯度下降时应对mini_batch的可变性) 列数为3072
x = tf.placeholder(tf.float32, [None, 3072])
# 为label提供占位符 数据类型离散 
y = tf.placeholder(tf.int64, [None])

#get_variable() 当已经定义了名字为w的变量时则使用已经定义的w 否则新定义w
#与每个样本做内积 大小[3072, 1]为每个样本的维度3072 * 二分类器输出为 1 初始化使用通用方式-正态分布 均值为0 方差为1
w = tf.get_variable('w', [x.get_shape(-1), 1],
                   initializer=tf.random_normal_initializer(0, 1))
#偏置 初始化为0.0
b = tf.get_variable('b', [1],
                   initializer=tf.constant_initializerp(0.0))

#[None, 3072] * [3072, 1] = [None, 1]
y_ = tf.matmul(x, w) + b # y为内积值

#[None, 1]
p_y_1 = tf.nn.sigmoid(y_) # 将y映射为 0-1 的概率值
#将y的大小转为[None, 1]
y_reshaped = tf.reshape(y, (-1, 1))
y_reshaped_float = tf.cast(y_reshaped, tf.float32)

#计算损失值 真实值与预测值的差的平方的均值
loss = tf.reduce_mean(tf.square(y_reshaped_float - p_y_1))

#bool [None, 1]
predict = p_y_1 > 0.5
#bool [None, 1] 转为int后为 [0, 1, 0, 1, 1, 0, 0]
correct_prediction = tf.equal(tf.cast(predict, tf.int64), y_reshaped)
#正确率 求均值
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

#adam为梯度下降的变种 初始化的learning_rate为10^-3 
with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss) 
